# 16. RNN과 어텐션을 사용한 자연어 처리

In [1]:
# 공통 모듈 임포트
import numpy as np
import os

# 깔금한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

## 16.1 Char-RNN으로 셰익스피어 문체 생성하기

### 16.1.1 훈련 데이터셋 만들기

In [2]:
import tensorflow as tf

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

shakespeare_url = 'https://homl.info/shakespeare'
filepath = tf.keras.utils.get_file('shakespeare.txt', shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [3]:
print(shakespeare_text[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


In [4]:
text_vec_layer = tf.keras.layers.TextVectorization(split = 'character', standardize='lower')
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]

2023-10-30 01:35:54.540027: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-10-30 01:35:54.540054: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-10-30 01:35:54.540062: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-10-30 01:35:54.540294: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-30 01:35:54.540318: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-10-30 01:35:54.667527: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [5]:
encoded -= 2
n_tokens = text_vec_layer.vocabulary_size() - 2
dataset_size = len(encoded)

In [6]:
def to_dataset(sequence, length, shuffle = False, seed = None, batch_size = 32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift = 1, drop_remainder = True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(buffer_size = 100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [10]:
length = 100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000], length=length, shuffle=True, seed = 42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], length=length)
test_set = to_dataset(encoded[1_060_000:], length=length)

### 16.1.2 Char-RNN 모델 만들고 훈련하기

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation = 'softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
model_ckpt = tf.keras.callbacks.ModelCheckpoint('my_shakespeare_model', monitor = 'val_accuracy', save_best_only=True)
history = model.fit(train_set, validation_data= valid_set, epochs=2, callbacks=[model_ckpt])

Epoch 1/2


2023-10-30 01:41:03.227740: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-30 01:41:10.656082: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-30 01:41:10.759386: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


  31247/Unknown - 755s 24ms/step - loss: 1.9545 - accuracy: 0.4181

2023-10-30 01:53:37.631505: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 1045853850338730159
2023-10-30 01:53:37.631687: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 17037774049988833223
2023-10-30 01:53:37.631693: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 9007514600193704349
2023-10-30 01:53:37.631753: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 8800234887924680155
2023-10-30 01:53:37.631761: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 3072504827153520709
2023-10-30 01:53:37.631766: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 6276434334016941993
2023-10-30 01:53:37.631919: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv it

INFO:tensorflow:Assets written to: my_shakespeare_model/assets


INFO:tensorflow:Assets written to: my_shakespeare_model/assets


31247/31247 [==============================] - 776s 25ms/step - loss: 1.9545 - accuracy: 0.4181 - val_loss: 1.9396 - val_accuracy: 0.4209
Epoch 2/2
21776/31247 [===================>..........] - ETA: 3:53 - loss: 1.8806 - accuracy: 0.4391

In [ ]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),
    model
])

In [ ]:
y_proba = shakespeare_model.predict(['To be or not to b'])[0, -1]
y_pred = tf.argmax(y_proba)
text_vec_layer.get_vocabulary()[y_pred + 2]

### 16.1.3 가짜 셰익스피어 텍스트 생성하기

In [ ]:
def next_char(text, temperature = 1):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logit = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logit, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [ ]:
def extend_text(text, n_chars = 50, temperature = 1):
    for _ in range(n_chars):
        text += next_char(text, temperature)

    return text

In [ ]:
tf.random.set_seed(42)

In [ ]:
extend_text('To be or not to be', temperature=0.01)

In [ ]:
extend_text('To be or not to be', temperature=1)

In [ ]:
extend_text('To be or not to be', temperature=100)

### 16.1.4 상태가 있는 RNN